## Dynamic Views: Restricting data to a subset based on a field:

Note: using dynamic view was the solution before Row level and column-level masking with SQL FUNCTIONS.

In [0]:
%run ./_resources/00-setup

In [0]:
-- As ANALYST from the USA (ANALYST_USA group), each USA row are now at "true":
SELECT is_account_group_member(group_name), * 
FROM (SELECT CONCAT("ANALYST_", country) AS group_name, country, id, firstname 
      FROM customers)

In [0]:
CREATE OR REPLACE VIEW customer_dynamic_view AS (
  SELECT *
  FROM customers
  WHERE is_account_group_member(CONCAT('ANALYST_', country))
);

-- Then grant select access on the view only:
GRANT SELECT ON VIEW customer_dynamic_view TO `account users`;

In [0]:
-- We should be part of the ANALYST_USA group. As result, we now have a row-level filter applied in our secured view and we only see the USA country:
SELECT * FROM customer_dynamic_view;

## Data Masking:

Let's create a table with all our current analyst permission including a GDPR permission flag: `analyst_permissions`.

This table has 3 field:

* `analyst_email`: to identify the analyst (we could work with groups instead).
* `country_filter`: we'll filter the dataset based on this value.
* `gdpr_filter`: if true, we'll filter the PII information from the table. If not set the user can see all the information.

*Of course this could be implemented with the previous `is_account_group_member()` function instead of individual users information being saved in a permission tale.*

In [0]:
SELECT * 
FROM analyst_permissions 
WHERE analyst_email = current_user();

In [0]:
CREATE OR REPLACE VIEW customer_dynamic_view_gdpr AS (
  SELECT id, creation_date, country, gender, age_group,
         CASE WHEN p.gdpr_filter = 1 THEN sha1(firstname) ELSE firstname END AS firstname,
         CASE WHEN p.gdpr_filter = 1 THEN sha1(lastname) ELSE lastname END AS lastname,
         CASE WHEN p.gdpr_filter = 1 THEN sha1(email) ELSE email END AS email
  FROM customers c
  INNER JOIN analyst_permissions p
  ON c.country = p.country_filter
  WHERE p.analyst_email = current_user()
);

-- Then grant select access on the view only:
GRANT SELECT ON VIEW customer_dynamic_view_gdpr TO `account users`;

In [0]:
SELECT * FROM customer_dynamic_view_gdpr;